In [ ]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from string import punctuation

import matplotlib.pyplot as plt
%matplotlib inline

import praw
import pprint
from datetime import datetime
import time

In [ ]:
reddit = praw.Reddit(user_agent='Comment Extraction (by /u/Extension_Dingo_1388)',
                     client_id='B6xLdwKHWNW_TML-bSy4Hw', client_secret="VGTupPwaa12dBchyIvX9nqLiioZj6A",
                     username='Extension_Dingo_1388', password='Music@22')

In [ ]:
list_of_subreddits = ['NFT', 'OpenSea', 'NFTExchange', 'CryptoArt', 'Decentraland', 'NFTmarket', 'NFTstocks', 'Rarible', 'Foundation', 'CardanoNFTs', 'SolanaNFTs' ]

In [ ]:
for i in list_of_subreddits:
    print("program started", datetime.now())
    start_time = time.time()
    posts = []
    sa_subreddit = reddit.subreddit(i)
    error_list = []
    error_count = 0

    for post in sa_subreddit.top(limit=None):
        try:
            date_created = datetime.utcfromtimestamp(post.created_utc)
            if date_created.year <2021:
                print("entered year break")
                break #check if its break/continue
            posts.append([post.subreddit, post.title, post.selftext, post.id, date_created, post.author, post.link_flair_richtext, post.permalink, post.ups, post.downs, post.num_comments,post.score , post.total_awards_received, post.id,  "post"])
            post.comments.replace_more(limit=0) #goes through all the comments without stopping at the 'more comment'
            for comment in post.comments.list():
                date_created_comment = datetime.utcfromtimestamp(comment.created_utc)
                posts.append([comment.subreddit, "", comment.body, comment.id, date_created_comment, comment.author, comment.author_flair_richtext, comment.permalink, comment.ups, comment.downs, "", comment.score , comment.total_awards_received, comment._submission, "comment"])
        except Exception as e :
            error_count += 1
            #print("layer 1" ,e)
            try:
                error_list.append([post.permalink, comment.permalink, comment.id])
            except:
                #print("layer 2" ,e)
                try:
                    error_list.append([post.permalink, post.id])
                except:
                    #print("layer 3" ,e)
                    pass

    print("error count is", error_count,)
    print("run time is", time.time()-start_time)
    
    print(i, "num posts is" , len(posts))
    posts_df = pd.DataFrame(posts,columns=['subreddit', 'title', 'selftext', 'id', 'date_created', 'author', 'link_flair_richtext', 'permalink', 'ups', 'downs', 'num_comments', 'score' , 'total_awards_received', 'submission', 'p or c'])
    posts_df.to_csv("dataset/"+str(i)+"_top.csv",index = False)


In [14]:
import os
import pandas as pd
checklist = ['NFTmarket_top.csv',
 'Foundation_top.csv',
 'NFTstocks_top.csv',
 'Rarible_top.csv',
 'OpenSea_top.csv',
 'CardanoNFTs_top.csv',
 'NFTArt_Finance_top.csv',
 'Decentraland_top.csv',
 'NFT_top.csv',
 'NFTExchange_top.csv',
 'CryptoArt_top.csv',
 'SolanaNFTs_top.csv']
# newlist=[]

def stackcsv(content_folder):
    global combined_csv
    combined_csv= []
    entries = os.listdir(content_folder)
    for i in entries:
        if i in checklist:
#             newlist.append(i)
            csv_path = os.path.join(content_folder, i)
            solo_csv = pd.read_csv(csv_path,index_col=None,header = None, on_bad_lines='skip')
            combined_csv.append(solo_csv)
#     return newlist  
    csv_final = pd.concat(combined_csv,axis = 0,sort = False)
    return csv_final.to_csv("combined_top_data.csv", header = None)

stackcsv('dataset')

/var/folders/39/d8tcqgqj7mv7xyp9q2m1m3x80000gn/T/ipykernel_39436/2681320716.py:25: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  solo_csv = pd.read_csv(csv_path,index_col=None,header = None, on_bad_lines='skip')


In [20]:
import csv

checklist = ['NFTmarket_top.csv',
 'Foundation_top.csv',
 'NFTstocks_top.csv',
 'Rarible_top.csv',
 'OpenSea_top.csv',
 'CardanoNFTs_top.csv',
 'NFTArt_Finance_top.csv',
 'Decentraland_top.csv',
 'NFT_top.csv',
 'NFTExchange_top.csv',
 'CryptoArt_top.csv',
 'SolanaNFTs_top.csv']
x = 0
for i in checklist:
    inputfile=open("dataset/"+str(i), "r+")
    reader_file = csv.reader(inputfile)
    value = len(list(reader_file))
    print(value , i)
    x+= value
print(x)
    

644 NFTmarket_top.csv
4085 Foundation_top.csv
1187 NFTstocks_top.csv
597 Rarible_top.csv
55750 OpenSea_top.csv
7475 CardanoNFTs_top.csv
10487 NFTArt_Finance_top.csv
8315 Decentraland_top.csv
16382 NFT_top.csv
30032 NFTExchange_top.csv
3839 CryptoArt_top.csv
218 SolanaNFTs_top.csv
139011


In [21]:
inputfile=open("combined_top_data.csv", "r+")
reader_file = csv.reader(inputfile)
value = len(list(reader_file))
print(value)

325133


references:

https://stackoverflow.com/questions/60489485/stackig-vertically-csv-files-with-pandas-in-python 
